In [ ]:
def non_max_suppression(boxes, probs=None, overlapThresh=0.65):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    if probs is not None:
        idxs = probs

    idxs = np.argsort(idxs)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")


In [ ]:
import cv2

# Check for available camera devices
available_cameras = []
for i in range(10):  # Check indices 0-9
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera index {i} is available.")
        available_cameras.append(i)
        cap.release()
    else:
        print(f"Camera index {i} is not available.")

print("Available cameras:", available_cameras)


In [5]:
import numpy as np
import cv2

# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

# Open webcam video stream
cap = cv2.VideoCapture(1)

# Output will be written to output.avi
out = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640, 480))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
    
    # Using a grayscale picture, also for faster detection
    # gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray = frame # setting to colour for now
    
    # Detect people in the image
    # Returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05, useMeanshiftGrouping=False)

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    
    for (xA, yA, xB, yB) in boxes:
        # Display the detected boxes in the color picture
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
    # Write the output video
    out.write(frame.astype('uint8'))
    
    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()

# And release the output
out.release()


# Finally, close the window1q
cv2.destroyAllWindows()
cv2.waitKey(1)


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

In [ ]:
# Your existing HOG code
# ...

# Non-max suppression
boxes, weights = hog.detectMultiScale(frame, winStride=(8, 8))
if len(boxes) > 0:
    pick = non_max_suppression(boxes, probs=weights, overlapThresh=0.65)
    for (xA, yA, xB, yB) in pick:
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)


In [ ]:
import numpy as np
import cv2

# Faster Non-Maximum Suppression function by Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
    return boxes[pick].astype("int")

# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

# Open webcam video stream (try different indices if this doesn't work)
cap = cv2.VideoCapture(0)

# # Output will be written to output.avi
# out = cv2.VideoWriter(
#     'output.avi',
#     cv2.VideoWriter_fourcc(*'MJPG'),
#     15.,
#     (640, 480))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # # Resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
    
    # Using a grayscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Detect people in the image
    boxes, weights = hog.detectMultiScale(frame, winStride=(8, 8))
    
    # Apply non-max suppression to remove multiple boxes around the same person
    if len(boxes) > 0:
        pick = non_max_suppression_fast(boxes, overlapThresh=0.65)
        
        if len(pick) > 0:
            xA = min(pick[:, 0])
            yA = min(pick[:, 1])
            xB = max(pick[:, 2])
            yB = max(pick[:, 3])
            cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
    # # Write the output video
    # out.write(frame.astype('uint8'))
    
    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and the output
cap.release()
# out.release()

# Finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)
